In [91]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models

# Define paths and parameters
train_images_dir = 'D:/Data science repo/Weapons_dataset/weapon_detection/train/images'
img_height, img_width = 224, 224
batch_size = 32

# Function to extract the class label from the filename
def extract_label_from_filename(image_file):
    base_name = tf.strings.split(image_file, os.sep)[-1]
    weapon_name = tf.strings.split(base_name, '_')[0]
    return weapon_name

# Create a mapping from weapon names to class IDs
image_files = [os.path.join(train_images_dir, f) for f in os.listdir(train_images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
weapon_names = sorted(set(extract_label_from_filename(tf.constant(f)).numpy().decode('utf-8') for f in image_files))
class_mapping = {name: i for i, name in enumerate(weapon_names)}

# Convert class_mapping to a TensorFlow lookup table
table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(weapon_names),
        values=tf.constant(list(range(len(weapon_names))), dtype=tf.int32)
    ),
    default_value=tf.constant(-1)
)

# Function to load and preprocess the image and extract the label
def load_image_and_label(image_file):
    img = tf.io.read_file(image_file)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, [img_height, img_width])
    img = tf.cast(img, tf.float32) / 255.0
    
    # Extract the weapon class from the filename
    weapon_name = extract_label_from_filename(image_file)
    class_id = table.lookup(weapon_name)
    
    return img, class_id

# Create a dataset from image files
def generator():
    for image_file in image_files:
        yield load_image_and_label(image_file)

dataset = tf.data.Dataset.from_generator(generator, output_signature=(
    tf.TensorSpec(shape=(img_height, img_width, 3), dtype=tf.float32),
    tf.TensorSpec(shape=(), dtype=tf.int32)
))
dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Load MobileNetV2 model with pre-trained weights
base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(img_height, img_width, 3)
)

# Freeze the base model
base_model.trainable = False

# Create a new model on top of MobileNetV2
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(len(class_mapping), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Check dataset shapes
for images, labels in dataset.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)

# Train the model
epochs = 100
history = model.fit(dataset, epochs=epochs)


Images shape: (32, 224, 224, 3)
Labels shape: (32,)
Epoch 1/100
18/18 [==============================] - 16s 823ms/step - loss: 4.4028 - accuracy: 0.0736
Epoch 2/100
18/18 [==============================] - 15s 812ms/step - loss: 2.0421 - accuracy: 0.2504
Epoch 3/100
18/18 [==============================] - 15s 807ms/step - loss: 2.0407 - accuracy: 0.2785
Epoch 4/100
18/18 [==============================] - 14s 797ms/step - loss: 1.6050 - accuracy: 0.4431
Epoch 5/100
18/18 [==============================] - 14s 782ms/step - loss: 1.3554 - accuracy: 0.5394
Epoch 6/100
18/18 [==============================] - 14s 770ms/step - loss: 1.2141 - accuracy: 0.5972
Epoch 7/100
18/18 [==============================] - 14s 775ms/step - loss: 1.0764 - accuracy: 0.6550
Epoch 8/100
18/18 [==============================] - 14s 798ms/step - loss: 0.9681 - accuracy: 0.6988
Epoch 9/100
18/18 [==============================] - 14s 792ms/step - loss: 0.8778 - accuracy: 0.7408
Epoch 10/100
18/18 [==========

In [93]:
def preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, [img_height, img_width])
    img = tf.cast(img, tf.float32) / 255.0
    img = tf.expand_dims(img, axis=0)  # Add batch dimension
    return img

In [94]:
# Assuming the model is already compiled and trained
def predict_image(image_path):
    # Load and preprocess the image
    img = preprocess_image(image_path)
    
    # Make prediction
    predictions = model.predict(img)
    
    # Get the predicted class ID
    predicted_class_id = tf.argmax(predictions[0]).numpy()
    
    # Map the class ID to the class label
    predicted_class_label = [name for name, id in class_mapping.items() if id == predicted_class_id][0]
    
    return predicted_class_label


In [101]:
sample_image_path = 'bazooka.jpeg'
predicted_label = predict_image(sample_image_path)
print(f"The predicted label for the sample image is: {predicted_label}")


1/1 [==============================] - 0s 41ms/step
The predicted label for the sample image is: Bazooka


In [98]:
model.save('pixelpunk.h5')

d:\Python\hvenv2\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
